In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns

from sklearn.linear_model import LinearRegression, LassoCV, RidgeCV, ElasticNet
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.preprocessing import StandardScaler

from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import RandomForestRegressor

%matplotlib inline

# Table of content
- [Data Preprocessing](#data-preprocessing)
    - [GDP-Major Industry from 1997 to 2018](#gdp-major-97-18)
    - [Trade Balance](#trade-balance)
        - [Import](#import)
        - [Export](#export)
    - [Investment](#investment)
        - [Investment to abroad](#inv-to-abroad)
        - [Investment from abroad](#inv-from-abroad)
    - [US Population](#us-population)
    - [Empolyment](#employment)
- [Merge Data](#merge)
- [Exploratory Data Analysis](#eda)
- [Feature Selection](#feature-selection)
    - [One Feature](#one-feature)
    - [Other Feature Tested by Linear Regression](#other-features)
- [Model Selection](#model-selection)
    - [Three Features Model](#3-features)
    - [Five Features Model](#5-features)
- [Conclusion](#conclusion)
- [Resource](#resource)

# Data Preprocessing<a name="data-preprocessing"></a> 

In order to predict the GDP, I did some study and decided which features may affect the GDP of Manufacturing Industry and make sure I can gather them from the government open source websites. Thus, the data were from multiple sources with diversity formats. In this case, I preprocessed all the data and changed the index to be equal to the year. Then I merged all the data together for exploring data analysis.

### GDP-Major Industry from 1997 to 2018<a name="gdp-major-97-18"></a>

Historical GDP data is from the US Bureau of Economic Analysis. The data is the GDP of all major industries between 1997 to 2018. The following step is the preprocess. First of all, the "U.Value by Industry" column contains information such as the data frame source, units (billions of dollars) and index lines. I don't want this information show in my data, so I dropped the column. Then, I checked for missing data and delete multiple rows with no information. I want the index to be years, so I set the name of the column equal to the year. Use a row to convert the column and reset the index to year. Finally, I renamed the column name and added a source to indicate its source.

In [2]:
# Load the data
gdp_97_18 = pd.read_excel('./data/Gross_domestic_product(GDP).xls')

FileNotFoundError: [Errno 2] No such file or directory: './data/Gross_domestic_product(GDP).xls'

In [ ]:
# Check the head of the data. The unit is Millions of current dollars.
gdp_97_18.head(5)

In [ ]:
# Select the Total GDP and Manufacturing
gdp_97_18 = gdp_97_18.loc[[5,16]]

In [ ]:
# Remove the first 3 columns.
gdp_97_18 = gdp_97_18.drop(columns = ['SAGDP2N Gross domestic product (GDP) by state 1/',
                         'Unnamed: 1','Unnamed: 2'])

In [ ]:
# Rename the columns to year.
gdp_97_18.columns = ['Year',1997,1998,1999,2000,2001,2002,2003,2004,
                     2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,
                    2015,2016,2017,2018]

In [ ]:
# Convert the row with column.
gdp_97_18 = gdp_97_18.T

In [ ]:
# Rename the columns with total gdp and manufacturing. 
gdp_97_18.columns = ['total GDP','manufacturing GDP']

In [ ]:
# Remove the first row.
gdp_97_18 = gdp_97_18[1:]

In [ ]:
gdp_97_18_m = gdp_97_18[['manufacturing GDP']]

In [ ]:
gdp_97_18_m = gdp_97_18_m*1_000_000

# Trade Balance<a name='trade-balance'></a>

## Import<a name ='import'></a>

The import data has detail information of all the country contribute to the import between 2002 to 2018. Remove the first two no information rows, and the bottom rows with the note of the data. Reset the column name to year, and then convert the columns with rows for merging.

In [ ]:
# load import data
import_02_18 = pd.read_excel('./data/Import_manuf_2002_2018.xlsx')
import_02_18.head()

In [ ]:
# drop the frist 2 rows
import_02_18 = import_02_18[3:241]

In [ ]:
# Rename the column
new_col_list =['country_import']
for i in range(2002,2019):
    new_col_list.append(i)

In [ ]:
# Rename the column
import_02_18.columns = new_col_list

In [ ]:
# Convert the row and column
import_02_18t = import_02_18.T

In [ ]:
# convert the first row to column name
import_02_18t.columns = [import_02_18t.iloc[0]]

In [ ]:
# Select the first level as the column name 
import_02_18t.columns = import_02_18t.columns.get_level_values(0)

In [ ]:
# Drop the first row
import_02_18t = import_02_18t[1:]

In [ ]:
# Check the shape of clean import data
import_02_18t.shape

In [ ]:
# Rename world to total_import
import_02_18t = import_02_18t.rename(columns ={'World':'total_import'})

## Export <a name='export'></a>

The export data has detail information about the detail industries contribute to the export between 2002 to 2018. Reset the column name to year, and then convert the columns with rows. Remove the column with the note of data and columns with no information.

In [ ]:
# load the export data
export_02_18 = pd.read_excel('./data/Export_manuf_2002_2018.xlsx')

In [ ]:
# Check the data 
export_02_18.head()

In [ ]:
# Convert the column and row
export_02_18t = export_02_18[:26].T

In [ ]:
# Rename the column
export_02_18t.columns = [export_02_18t.iloc[0]]

In [ ]:
# Drop the first row
export_02_18t = export_02_18t[1:]

In [ ]:
# Check the data a
export_02_18t.describe()

In [ ]:
# Drop the last three uninformation columns
export_02_18t = export_02_18t.drop(columns = ['3ZZ--MANUFACTURING PRODUCTS, NESOI',
                             '33Z--MANUFACTURING PRODUCTS, PART 3, NESOI',
                             'Provided by the Office of Trade and Economic Analysis (OTEA), Industry and Analysis, International Trade Administration, U.S. Department of Commerce'])


In [ ]:
# Create a list of column name from multindex 
# and set the string to lowercase
col_list = [ind.lower() for ind in export_02_18t.columns.get_level_values(0)]


In [ ]:
# Rename the column name
export_02_18t.columns = col_list

In [ ]:
# Rename the total to total_export
export_02_18t = export_02_18t.rename(columns = {'total':'total_export'})

In [ ]:
# Check the shape of export data
export_02_18t.shape

In [ ]:
# Merge the import with export.
balance = import_02_18t.merge(export_02_18t, how = 'inner',
                             left_index = True, right_index = True)

In [ ]:
balance['trade_balance'] = balance['total_export'] - balance['total_import']

# Investment <a name='investment'></a>

## Investment to abroad  <a name='inv-to-abroad'></a>

I selected the balance of payments and direct investment abroad data to combine with foreign direct investment data. Calculate the   as total investment 

In [ ]:
# Load teh investment direct to abroad data
inv_to_abroad = pd.read_excel('./data/Pharmaceutical_investment_us_to_other.xls')

In [ ]:
# Check the data
inv_to_abroad.head(7)

In [ ]:
# Select the data size
inv_to_abroad = inv_to_abroad[6:7]

In [ ]:
# Drop the first row
inv_to_abroad = inv_to_abroad.drop(columns = 'Balance of Payments and Direct Investment Position Data')

In [ ]:
# Change the column's name to year
inv_to_abroad.columns = [i for i in range(1999, 2018)]

In [ ]:
# Convert the row with column
inv_to_abroadt = inv_to_abroad.T

In [ ]:
# Rename the column 
inv_to_abroadt.columns = ['manufacturing_inv_to_abroad']

In [ ]:
inv_to_abroadt.head()

## Investment from abord  <a name='inv-from-abroad'></a>

In [ ]:
# Load the investment from abroad
inv_from_abroad = pd.read_excel('./data/Pharmaceutical_investment_other_to_us.xls')


In [ ]:
# Check the investment to abroad data
inv_from_abroad.head(7)

In [ ]:
# Select the data size
inv_from_abroad = inv_from_abroad[6:7]

In [ ]:
# Drop the first row
inv_from_abroad = inv_from_abroad.drop(columns = 'Balance of Payments and Direct Investment Position Data')


In [ ]:
# Change the column's name from year
inv_from_abroad.columns = [i for i in range(1997, 2018)]

In [ ]:
# Convert the row with column
inv_from_abroadt = inv_from_abroad.T

In [ ]:
# Rename the column 
inv_from_abroadt.columns = ['manufacturing_inv_from_abroad']

inv_from_abroadt.head()

In [ ]:
# Merge both the investment data.
inv = inv_to_abroadt.merge(inv_from_abroadt, how = 'inner',
                         left_index = True, right_index = True)

In [ ]:
# Create a total investment column in the merge dataframe.
inv['inv'] = inv['manufacturing_inv_from_abroad']-inv['manufacturing_inv_to_abroad']

## US Population <a name="us-population"></a>

In [ ]:
# Load the population data.
pop= pd.read_excel('./data/POP.TOTL_all_country.xls')

In [ ]:
# Check the data
pop.head()

In [ ]:
# Create a list of number from 1960 to 2018
year_list_60_18 = np.arange(1960,2019)
year_list_60_18

In [ ]:
# Rename the column
pop_new_col = ['Country Name', 'Country Code','Indicator Name',
              'Indicator Code']

In [ ]:
# Append the year list to pop_new_col
for year in year_list_60_18:
    pop_new_col.append(year)
pop_new_col[0:7]

In [ ]:
# Rename the column of pop. 
pop.columns = pop_new_col

In [ ]:
# Check the shape of pop data.
pop.shape

In [ ]:
# Check the missing value of year 2018
pop[2018].isna().sum()

In [ ]:
# Drop the year 2018 due to uninfomative. 
pop.drop(columns = [2018], inplace = True)

In [ ]:
# I wanted the data between 2000 to 2018 only,
# so I droped the column from 1960 to 1999
# create a list for the above years
year_list_60_99 = [year for year in range(1960,2000)]
year_list_60_99[-5:]

In [ ]:
# Drop the years in columns
pop = pop.drop(columns = year_list_60_99)

In [ ]:
# Select the USA as data only
mask2 = pop['Country Name'] == 'United States'
us_pop = pop[mask2]

In [ ]:
# Since we need only year data with population,
# I removed the first 4 columns, too.
us_pop = us_pop.drop(columns = ['Country Name','Country Code',
                       'Indicator Name','Indicator Code'])

In [ ]:
# Convert the row and columns.
us_pop = us_pop.T

In [ ]:
# Rename the column name.
us_pop.columns = ['us population']

In [ ]:
us_pop.head()

## Employment <a name='employeement'></a>

I considered the number of employee as one of the feature of the cost in GDP. Thus, I collected the data from the U.S. Bureau of Labor Statistics and select the three categories of employee number, include total, manufacturing, pharmaceutical and medicine manufacturings. 

In [ ]:
# Load the employeement data
emp = pd.read_excel('./data/Employment_us.xlsx')

In [ ]:
# Take a look at emp data
emp.head()

In [ ]:
# Convert the year to be index
emp = emp.T

In [ ]:
# Rename the column name 
emp.columns = ['total employee','manufacturing employee',
               'p&m employee']

In [ ]:
# Drop the first row
emp = emp[1:]

In [ ]:
# Times a thoudsand to get the correct employment number
emp = emp*1000

In [ ]:
# Add the 
emp_m = emp[['manufacturing employee']]

In [ ]:
emp_m.head()

# Merge Data <a name='merge'></a>

In [ ]:
# merge the data by emp
gdp_emp = gdp_97_18_m.merge(emp_m, how ='right',
                 left_index=True, right_index=True)

In [ ]:
# Merge the gdp_emp with population
gdp_emp_pop = gdp_emp.merge(us_pop, how = 'inner',
                 left_index = True,
                 right_index = True)

In [ ]:
# Merge the balance with total investment.
balance_inv = balance.merge(inv,  how = 'inner',
                             left_index = True, right_index = True)

In [ ]:
# Merge all the data together
df = gdp_emp_pop.merge(balance_inv, how = 'inner',
                       left_index = True, right_index = True)

# Exploratory Data Analysis <a name='eda'></a>

In [ ]:
# Check the df data
df.head()

In [ ]:
# Check the data shape
df.shape

In [ ]:
# Check the null value 
null = df.isnull().sum()

In [ ]:
# Check the column name
df.columns.values[0:5]

In [ ]:
# Check the type of each columns
df.info()

In [ ]:
# Change the type of each column to int
df = df.astype(float)

In [ ]:
# Set the figure size
plt.figure(figsize = (3,50))

# Some of the columns were not showing, so plot the bar plot
sns.heatmap(df.corr()[['manufacturing GDP']].sort_values(['manufacturing GDP']),
           annot = True, cmap = 'coolwarm')

In [ ]:
df.info()

In [ ]:
# the features are too much, so I shark the feature to only 6
df_shark = df[['manufacturing GDP','manufacturing employee',
              'us population','inv','trade_balance','total_import','total_export']]

In [ ]:
# Change the columns name of shark 
df_shark.columns = ['GDP','employee','population','inv','trade_balance','import','export']

In [ ]:
df_shark['year'] = df_shark.index.astype(float)

In [ ]:
# Check the type of df_shark data
df_shark.info()

In [ ]:
# Set the figure size
plt.figure(figsize = (2,3))
# Plot the heatmap to check the correlation of each feature
sns.heatmap(df_shark.corr()[['GDP']].sort_values('GDP'),
            annot = True, cmap = 'coolwarm')

In [ ]:
plt.figure(figsize = (8,5))
# I want to know how each feature growth each year
features = ['GDP','import','export','trade_balance']

sns.lineplot('year','GDP', data = df_shark)
sns.lineplot('year','import', data = df_shark)
sns.lineplot('year','export', data = df_shark)
sns.lineplot('year','trade_balance',data =df_shark)

plt.xlabel('Year')
plt.ylabel('Value Add')
plt.legend(labels = features)
plt.title('Impot/Export and GDP Growth Value Since 2002')

- The line chart in the above chart shows the relationship between GDP and imports, exports and trade balance over the years. First, since 2009, except for 2009, the growth trend of GDP and total imports has increased. The huge decline in 2009 was due to the financial crisis that began in the United States and the global economic recession that caused the world's economic recession. Export trends fell twice in 2009 and 2015 to 2016. For the year 2015 to 2016, the weak global economy and the strengthening of the US dollar, exports have fallen, making US goods and services more expensive. Second, the growth rate of import growth has exceeded the trend before 2008. 

In [ ]:
plt.figure(figsize = (9,4))
# plot the bar plot
sns.barplot(df_shark['year'].astype(int),'population', data = df_shark)

# Set the labels
plt.xlabel('Year')
plt.ylabel('Population')
plt.title('Population Since 2002')

In [ ]:
# Set the figure size
plt.figure(figsize = (9,4))

# Plot the bar plot
sns.barplot(df_shark['year'].astype(int),'inv', data = df_shark)

# Set the labels
plt.xlabel('Year')
plt.ylabel('Investment')
plt.title('Investment Since 2002')

In [ ]:
# Set the figure size
plt.figure(figsize = (9,4))

# Plot the bar plot
sns.barplot(df_shark['year'].astype(int),'employee', data = df_shark)

# Set the labels
plt.xlabel('Year')
plt.ylabel('Employee')
plt.title('Employment Since 2002')

# Features selection <a name='features-selection'></a>

During the feature selection, I selected only 'import' as my feature because it showed 0.98 correlated to GDP in the above heatmap. I used it to test which model worked the best. But Linear Regression, Lasso, and Ridge model worked pretty well and showed about 0.96 in train score, and 0.97 in test score which is much higher than I expeted. High R^2 score indicated that import can explain ~96% of variance in total variance. However, in the formula of GDP, import is only one of the variables. Due to the prefrence of import variable, I tested all other features in df_shark and compare their result by using Linear Regression. 

## One feature <a name='one-feature'></a>

### Import-Linear Regression

In [ ]:
# Select the X and y
X = df_shark[['import']]
y = df_shark['GDP']

In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
# Define a function of linear regression to avoid the duplicate work
def linear_reg(X_var,y_var):
    # train test split the data
    X_var_train, X_var_test, y_var_train,y_var_test = train_test_split(X_var,y_var,
                                                       random_state = 4)

    # Initiate model
    lr = LinearRegression()

    # fit the model
    model = lr.fit(X_var_train, y_var_train)

    # Get the predict 
    lr_y_var_hat_train = model.predict(X_var_train)
    lr_y_var_hat_test = model.predict(X_var_test)

    # Score the predict
    print("(lr)R^2 Train score: ",r2_score(y_var_train,lr_y_var_hat_train))
    print("(lr)R^2 Test score: ",r2_score(y_var_test, lr_y_var_hat_test))
    return

In [ ]:
linear_reg(X,y)

### Import-Lasso

In [ ]:
def lasso(X_var,y_var):
    # train test split the data
    X_var_train, X_var_test, y_var_train,y_var_test = train_test_split(X_var,y_var,
                                                       random_state = 4)
    
    # Standard Scale the train and test of X
    # use ss is to let every data become unitless 
    ss = StandardScaler()

    #use 
    ss.fit(X_var_train)
    X_var_train_ss = ss.transform(X_var_train)
    X_var_test_ss = ss.transform(X_var_test)
    # Create lasso regression with some possible alpha values
    lasso = LassoCV(alphas=[0.01,0.1,1,5,10,100,1000,2000,
                            3000,4000,5000,10000], cv =5)

    # Fit the linear regression
    lasso_model = lasso.fit(X_var_train_ss , y_var_train)

    #Generate predictions
    y_hat_lasso_train_var = lasso_model.predict(X_var_train_ss)
    y_hat_lasso_test_var = lasso_model.predict(X_var_test_ss)

    # calculate teh R^2 score
    print("(lasso)R^2 Train score: ",r2_score(y_var_train, y_hat_lasso_train_var))
    print("(lasso)R^2 Test score: ",r2_score(y_var_test, y_hat_lasso_test_var))

In [ ]:
lasso(X,y)

### Import-Ridge

In [ ]:
def ridge(X_var,y_var):
    # train test split the data
    X_var_train, X_var_test, y_var_train,y_var_test = train_test_split(X_var,y_var,
                                                       random_state = 4)
    
    # Standard Scale the train and test of X
    # use ss is to let every data become unitless 
    ss = StandardScaler()

    #use 
    ss.fit(X_var_train)
    X_var_train_ss = ss.transform(X_var_train)
    X_var_test_ss = ss.transform(X_var_test)
    
    # Create ridge regression with some possible alpha values
    ridge = RidgeCV(alphas=[0.01,1,5,10,100,1000,2000,
                            3000,4000,5000,10000], cv =5)

    # Fit the linear regression
    ridge_model = ridge.fit(X_var_train_ss , y_var_train)

    #Generate predictions
    y_hat_ridge_train_var = ridge_model.predict(X_var_train_ss)
    y_hat_ridge_test_var = ridge_model.predict(X_var_test_ss)

    # calculate teh R^2 score
    print("(ridge)R^2 Train score: ",r2_score(y_var_train, y_hat_ridge_train_var))
    print("(ridge)R^2 Test score: ",r2_score(y_var_test, y_hat_ridge_test_var))

In [ ]:
ridge(X,y)

### Import-Random Forest Regressor

In [ ]:
def random_forset(X_var,y_var):
    # train test split the data
    X_var_train, X_var_test, y_var_train,y_var_test = train_test_split(X_var,y_var,
                                                       random_state = 4)
    
    # Standard Scale the train and test of X
    # use ss is to let every data become unitless 
    ss = StandardScaler()

    #use 
    ss.fit(X_var_train)
    X_var_train_ss = ss.transform(X_var_train)
    X_var_test_ss = ss.transform(X_var_test)

    # Tune GridSearchCV and use param_grid to get the best grid
    pipe_params={
        'bootstrap':[True,False],
        'max_depth': [None,3,4,5],
        'min_samples_leaf':[1,2,3,4]
    }
    # Initiate random forest model in GridSearchCV
    gs = GridSearchCV(RandomForestRegressor(n_estimators = 100), param_grid = pipe_params, cv=3)

    # fit the model
    gs_model = gs.fit(X_var_train_ss, y_var_train)

    # Generate the predict.
    y_hat_var_rf_train = gs_model.predict(X_var_train_ss)
    y_hat_var_rf_test = gs_model.predict(X_var_test_ss)

    # Calculate the R^2 score.
    print("(rf)R^2 Train score: ",r2_score(y_var_train, y_hat_var_rf_train))
    print("(rf)R^2 Test score: ",r2_score(y_var_test, y_hat_var_rf_test))

In [ ]:
random_forset(X,y)

## Other Features Tested by Linear Regression <a name='other-features'></a>

In [ ]:
# Set the X to year
X1 = df_shark[['year']]
y = df_shark['GDP']

# Run the linear_reg funtion to get the R^2 score
linear_reg(X1,y)

In [ ]:
df_shark.columns

In [ ]:
# Set X to population
X2 = df_shark[['population']]

# Run the linear_reg funtion to get the R^2 score
linear_reg(X2,y)

In [ ]:
# Set X to inv_to
X3 = df_shark[['inv']]

# Run the linear_reg funtion to get the R^2 score
linear_reg(X3,y)

In [ ]:
# Set X to export
X4 = df_shark[['export']]

# Run the linear_reg funtion to get the R^2 score
linear_reg(X4,y)

In [ ]:
# Set X to employee
X5 = df_shark[['employee']]

# Run the linear_reg funtion to get the R^2 score
linear_reg(X5,y)

In [ ]:
# Set X to balance
X6 = df_shark[['trade_balance']]

# Run the linear_reg funtion to get the R^2 score
linear_reg(X6,y)

# Model Selection <a name='model-selection'></a>

## Three Features Model <a name='3-features'></a>

In [ ]:
# Set the X
X7 = df_shark[['trade_balance','inv','population']]

# Linear Regression
linear_reg(X7,y)

In [ ]:
# Lasso
lasso(X7,y)

In [ ]:
# Ridge
ridge(X7,y)

In [ ]:
# Random Forest
random_forset(X7,y)

## Five Features Model<a name='5-features'></a>

In [ ]:
# Set the X
X8 = df_shark[['trade_balance','year','inv','population','employee']]

In [ ]:
# Linear Regression
linear_reg(X8,y)

In [ ]:
# Lasso
lasso(X8,y)

In [ ]:
# Ridge
ridge(X8,y)

In [ ]:
# Random Forest
random_forset(X8,y)

# Conclusion <a name='conclusion'></a>

First, since 2009, except for 2009, the growth trend of GDP and total imports has increased. The huge decline in 2009 was due to the financial crisis that began in the United States and the global economic recession that caused the world's economic recession. Export trends fell twice in 2009 and 2015 to 2016. For the year 2015 to 2016, the weak global economy and the strengthening of the US dollar, exports have fallen, making US goods and services more expensive. Second, the ridge model with five features, which include 'trade_balance', 'year', 'inv', 'population', and 'employee', worked the best. It showed 99 percent of the GDP can be explained by the model.  

# Future Improvement <a name='future-improve'></a>

The GDP is so correlated to import, I don't why for now. But I do need to keep learning more about their relationship. 

# Resource <a name='resource'> </a>

- GDP: https://www.bea.gov/data/gdp/gdp-industry

- GDP Calculate: 
    - https://courses.lumenlearning.com/boundless-economics/chapter/comparing-real-and-nominal-gdp/
    - https://www.slideshare.net/syler333/balance-of-payments-46077319
    - https://www.thebalance.com/what-is-gdp-definition-of-gross-domestic-product-3306038

- GDP affect factors: https://blog.marketresearch.com/the-growing-pharmaceuticals-market-expert-forecasts-and-analysis

- investment from/to abroad: https://www.bea.gov/data/intl-trade-investment/direct-investment-country-and-industry

- Import/Export: https://www.census.gov/foreign-trade/index.html

- Employee: https://www.bls.gov/cps/aa2012/cpsaat18b.htm

- US population: https://data.worldbank.org/indicator/SP.POP.TOTL?locations=US
- Economic drop: 
    - https://www.marketwatch.com/story/us-exports-fall-in-2015-for-first-time-since-recession-2016-02-05
    - https://www.forbes.com/2009/01/14/global-recession-2009-oped-cx_nr_0115roubini.html#49d387e5185f